In [29]:
#r "nuget: Microsoft.Extensions.Hosting, 8.0.0"
#r "nuget: Microsoft.Extensions.Options.ConfigurationExtensions, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.17.1"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.17.1"
#r "nuget: Microsoft.SemanticKernel.Connectors.Postgres, 1.17.1-alpha"
#r "nuget: Microsoft.KernelMemory.Core, 0.71.240820.1"
#r "nuget: Microsoft.KernelMemory.MemoryDb.Postgres, 0.71.240820.1"

#r "nuget: OpenTelemetry.Exporter.OpenTelemetryProtocol"
#r "nuget: OpenTelemetry.Extensions.Hosting"
#r "nuget: OpenTelemetry.Instrumentation.Http"
#r "nuget: OpenTelemetry.Exporter.Console"

Installed Packages Microsoft.Extensions.Hosting, 8.0.0 Microsoft.Extensions.Options.ConfigurationExtensions, 8.0.0 Microsoft.KernelMemory.Core, 0.71.240820.1 Microsoft.KernelMemory.MemoryDb.Postgres, 0.71.240820.1 Microsoft.SemanticKernel, 1.17.1 Microsoft.SemanticKernel.Connectors.OpenAI, 1.17.1 Microsoft.SemanticKernel.Connectors.Postgres, 1.17.1-alpha OpenTelemetry.Exporter.Console, 1.9.0 OpenTelemetry.Exporter.OpenTelemetryProtocol, 1.9.0 OpenTelemetry.Extensions.Hosting, 1.9.0 OpenTelemetry.Instrumentation.Http, 1.9.0

✅Imported `IKernelMemory` as `memory`...

✅Imported `Kernel` as `kernel`...

✅Imported `IServiceProvider` as `services`...


In [30]:
#!import ./ServiceDefaults.cs
#!import ./OpenAiOptions.cs

using Microsoft.Extensions.Hosting;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.KernelMemory;
using OpenTelemetry;
using Microsoft.KernelMemory.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

#pragma warning disable SKEXP0010

HostApplicationBuilder builder = Host.CreateApplicationBuilder();

builder.Environment.EnvironmentName = "Development";

ServiceDefaults.ConfigureOpenTelemetry(builder);

var textGenerationOpetions = builder.Configuration
    .GetSection("TextGeneration")
    .Get<OpenAiOptions>();

var textEmbeddingOptions = builder.Configuration
    .GetSection("TextEmbeddingGeneration")
    .Get<OpenAiOptions>();

builder
    .Services.AddKernel()
    .AddAzureOpenAIChatCompletion(
        deploymentName: textGenerationOpetions.Deployment,
        endpoint: textGenerationOpetions.Endpoint,
        apiKey: textGenerationOpetions.ApiKey
    );

builder.Services.AddKernelMemory<MemoryServerless>(memoryBuilder =>
{
    memoryBuilder
        .WithPostgresMemoryDb(
            new PostgresConfig()
            {
                ConnectionString = builder.Configuration.GetConnectionString("memory-db")!
            }
        )
        .WithSemanticKernelTextGenerationService(
            new AzureOpenAIChatCompletionService(
                deploymentName: textGenerationOpetions.Deployment,
                endpoint: textGenerationOpetions.Endpoint,
                apiKey: textGenerationOpetions.ApiKey
            ),
            new SemanticKernelConfig()
        )
        .WithSemanticKernelTextEmbeddingGenerationService(
            new AzureOpenAITextEmbeddingGenerationService(
                deploymentName: textEmbeddingOptions.Deployment,
                endpoint: textEmbeddingOptions.Endpoint,
                apiKey: textEmbeddingOptions.ApiKey
            ),
            new SemanticKernelConfig()
        );
});

IHost host = builder.Build();

IKernelMemory memory = host.Services.GetRequiredService<IKernelMemory>();
Kernel kernel = host.Services.GetRequiredService<Kernel>();
IServiceProvider services = host.Services;

info: Microsoft.KernelMemory.Handlers.TextExtractionHandler[0]
      Handler 'extract' ready
info: Microsoft.KernelMemory.Handlers.TextPartitioningHandler[0]
      Handler 'partition' ready
info: Microsoft.KernelMemory.Handlers.SummarizationHandler[0]
      Handler 'summarize' ready
info: Microsoft.KernelMemory.Handlers.GenerateEmbeddingsHandler[0]
      Handler 'gen_embeddings' ready, 1 embedding generators
info: Microsoft.KernelMemory.Handlers.SaveRecordsHandler[0]
      Handler save_records ready, 1 vector storages
info: Microsoft.KernelMemory.Handlers.DeleteDocumentHandler[0]
      Handler 'private_delete_document' ready
info: Microsoft.KernelMemory.Handlers.DeleteIndexHandler[0]
      Handler 'private_delete_index' ready
info: Microsoft.KernelMemory.Handlers.DeleteGeneratedFilesHandler[0]
      Handler 'delete_generated_files' ready
info: Microsoft.KernelMemory.Handlers.GenerateEmbeddingsParallelHandler[0]
      Handler 'gen_embeddings_parallel' ready, 1 embedding generators
info: